# 전처리
data set
- 일반농가
    - identity.csv(아이덴티티 정보)
    - cropping_user.csv(작기 정보)
    - all_cultivate_df.csv(모든 작기에 대한 토마토 생육정보)
    - cropping/{facilityId}.csv(작기별 제어.... 정보) <- 해석 어려움,,
        - 제어정보(CR)
    - cultivate/{facilityId}.csv(작기별 생육 정보)★<-전처리 필요
        - 내부환경(EI)
        - 외부환경(EO)
        - 토양환경(EL)
        - 양액정보(NT)
- 우수농가
    - grwdatarqst.csv
    - envdatarqst.csv
    - prddatarqst.csv

In [166]:
import os
import pandas as pd
import numpy as np
from utils import getCode

## 작기별 환경정보

In [145]:
pf_0000227 = pd.read_csv("data/seasonenv/PF_0000227.csv", index_col=0)
pf_0000227

,fatrCode,facilityId,measDate,fldCode,sectCode,itemCode,senVal,ymd,cntCollect,num,totalRows,totalPage,currentPage
0,RP,PF_0000227_01,2017-01-01 00:00:00,FG,EO,80300,0.00,20170101,26,1,90376.0,91.0,1.0
1,WD,PF_0000227_01,2017-01-01 00:00:00,FG,EO,80300,48.96,20170101,26,2,NaN,NaN,NaN
2,IR,PF_0000227_01,2017-01-01 00:00:00,FG,EI,80300,0.00,20170101,26,3,NaN,NaN,NaN
3,HL,PF_0000227_01,2017-01-01 00:00:00,FG,EL,80300,91.38,20170101,26,4,NaN,NaN,NaN
4,TI,PF_0000227_01,2017-01-01 00:00:00,FG,EI,80300,13.20,20170101,26,5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90995,EL,PF_0000227_01,2017-12-08 13:00:00,FG,EL,80300,9.80,20171208,23,89996,NaN,NaN,NaN
90996,HI,PF_0000227_01,2017-12-08 13:00:00,FG,EI,80300,95.45,20171208,23,89997,NaN,NaN,NaN
90997,WD,PF_0000227_01,2017-12-08 13:00:00,FG,EO,80300,39.61,20171208,23,89998,NaN,NaN,NaN
90998,TL,PF_0000227_01,2017-12-08 13:00:00,FG,EL,80300,13.55,20171208,23,89999,NaN,NaN,NaN


## 작기별 제어 정보

In [138]:
file = pd.read_csv(root_path+'\\'+file_list[0], index_col=0)
file.head()

,statusCode,statusMessage,fatrCode,facilityId,measDate,fldCode,sectCode,itemCode,senVal,ymd,cntCollect,num,totalRows,totalPage,currentPage
0,0.0,NORMAL_CODE,CC01,PF_0000002_01,2016-03-09 20:00:00,FG,CR,80300,0.0,20160309,1,1,20391.0,21.0,1.0
1,NaN,NaN,CC01_2_L_C,PF_0000002_01,2016-03-10 03:00:00,FG,CR,80300,0.0,20160310,1,2,NaN,NaN,NaN
2,NaN,NaN,CC01,PF_0000002_01,2016-03-10 03:00:00,FG,CR,80300,0.0,20160310,2,3,NaN,NaN,NaN
3,NaN,NaN,CC03,PF_0000002_01,2016-03-10 03:00:00,FG,CR,80300,0.0,20160310,1,4,NaN,NaN,NaN
4,NaN,NaN,CC01,PF_0000002_01,2016-03-11 02:00:00,FG,CR,80300,0.0,20160311,1,5,NaN,NaN,NaN


In [40]:
def agg_set(series):
    return set(series)

In [83]:
codes_in_file = pd.DataFrame(columns=['facilityCode', 'sectCode', 'fatrCode'])

for file in file_list:
    file_df = pd.read_csv(root_path+'\\'+file, index_col=0)
    sectCode_set = file_df.groupby('fldCode')['sectCode'].agg(agg_set)[0]
    fatrCode_set = file_df.groupby('fldCode')['fatrCode'].agg(agg_set)[0]
    
    temp = pd.Series(data=[file[:-4], sectCode_set, fatrCode_set],
                     index=['facilityCode', 'sectCode', 'fatrCode'],
                     name=len(codes_in_file))
    codes_in_file = codes_in_file.append(temp)

In [85]:
codes_in_file

,facilityCode,sectCode,fatrCode
0,PF_0000002,{CR},"{CC03_1_L_S_1, CC03, CC01_1_L_S_3, CC01, CC03_..."
1,PF_0000004,{CR},"{CC01_1_R_S_4, CC01_1_R_S_1, CC01_1_R_S_3, CC0..."
2,PF_0000006,{CR},"{CC03_1_L_S_3, CC0803, CC03_1_R_S_4, CC03_1_R_..."
3,PF_0000007,{CR},"{CC03_1_R_C, CC03_1_L_C}"
4,PF_0000008,{CR},"{CC08, CC03}"
5,PF_0000010,{CR},"{CC03_1_L_S_1, CC03_1_R_C, CC03, CC01, CC01_1_..."
6,PF_0000012,{CR},"{CC03_1_L_S_1, CC03_1_L_S_4, CC03_1_L_S_3, CC0..."
7,PF_0000013,{CR},{CC1403}
8,PF_0000014,{CR},"{CC03_1_L_S_1, CC03_1_R_C, CC03_1_L_S_4, CC03_..."
9,PF_0000015,{CR},"{CC04, CC01_2_L_C, CC01}"


In [136]:
code_list = []
for row in range(len(codes_in_file)):
    code_list.extend(list(codes_in_file['fatrCode'][row]))
for code in list(set(code_list)):
    print(code, getCode(code))

CC03_1_L_S_3 좌측일중측창 닫힘값
CC03_1_R_S_4 우측일중측창 온도폭
CC03 측창
CC04 차광커튼
CC03_3_L_O 좌측삼중측창 작동여부
CC1802 배기팬작동여부
CC03_2_L_V 좌측이중측창 개도
CC2002 관수작동상태 (토경)작동여부
CC01_3_L_C 좌측삼중천창 작동상태
CC0403 차광커튼작동상태
CC0904 난방기설정정보
CC01_1_R_O 우측일중천창 작동여부
CC01_1_L_S_2 좌측일중천창 열림값
CC1403 CO2발생장치작동상태
CC01_1_R_C 우측일중천창 작동상태
CC03_2_R_V 우측이중측창 개도
CC0802 유동팬작동여부
CC03_2_R_C 우측이중측창 작동상태
CC0804_3 유동팬 작동습도
CC01_2_L_O 좌측이중천창 작동여부
CC01 천창
CC01_2_R_O 우측이중천창 작동여부
CC03_2_L_S_4 좌측이중측창 온도폭
CC01_2_L_C 좌측이중천창 작동상태
CC03_1_R_O 우측일중측창 작동여부
CC03_1_R_S_3 우측일중측창 닫힘값
CC01_1_L_S_3 좌측일중천창 닫힘값
CC01_1_L_S_1 좌측일중천창 설정온도
CC03_2_L_C 좌측이중측창 작동상태
CC01_1_R_S_1 우측일중천창 설정온도
CC03_2_R_S_3 우측이중측창 닫힘값
CC03_2_R_S_1 우측이중측창 설정온도
CC01_1_R_V 우측일중천창 개도
CC03_2_L_S_3 좌측이중측창 닫힘값
CC01_1_R_S_3 우측일중천창 닫힘값
CC03_1_R_C 우측일중측창 작동상태
CC0904_2 난방기 정지온도
CC0502 보온커튼작동여부
CC03_2_R_O 우측이중측창 작동여부
CC0804_2 유동팬 정지온도
CC01_2_R_C 우측이중천창 작동상태
CC01_2_R_S_1 우측이중천창 설정온도
CC0803 유동팬작동상태
CC03_1_R_S_2 우측일중측창 열림값
CC03_1_L_C 좌측일중측창 작동상태
CC0804_1 유동팬 작동온도
CC03_1_L_O 좌측일중측창 작동여부
CC03_3_R_O 우측삼중측창 작동